In [1]:
import io
from datetime import datetime, timedelta
from pydantic import BaseModel
from time import sleep

In [2]:
from IPython.display import clear_output
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
import berserk
import chess, chess.pgn

In [4]:
from dash import Dash, html, dcc

In [50]:
from __future__ import annotations
from typing import List, Optional
from pydantic import BaseModel

class User(BaseModel):
    name: str
    id: str

class GlobalAnalysis(BaseModel):
    inaccuracy: int
    mistake: int
    blunder: int
    acpl: int

class Player(BaseModel):
    user: User
    rating: int
    ratingDiff: Optional[int] = 0
    analysis: Optional[GlobalAnalysis] = None

class Players(BaseModel):
    white: Player
    black: Player

class Opening(BaseModel):
    eco: str
    name: str
    ply: int
        
class Judgment(BaseModel):
    name: str
    comment: str

class MoveAnalysis(BaseModel):
    mate: Optional[int]
    eval: Optional[int]
    best: Optional[str] = None
    variation: Optional[str] = None
    judgment: Optional[Judgment] = None

class Clock(BaseModel):
    initial: int
    increment: int
    totalTime: int

class GameModel(BaseModel):
    id: str
    rated: bool
    variant: str
    speed: str
    perf: str
    createdAt: datetime
    lastMoveAt: datetime
    status: str
    players: Players
    winner: Optional[str] = 'draw'
    opening: Optional[Opening] = None
    moves: str
    analysis: Optional[List[MoveAnalysis]] = None
    clock: Clock

## Functions

In [51]:
# Read PGN string into python-chess object
def read_pgn(game_str):
    pgn = io.StringIO(game_str)
    game = chess.pgn.read_game(pgn)
    return game

In [52]:
# Read python-chess object, play and display it in them on a board.
def play_game(game):
    board = game.board()
    for move in game.mainline_moves():
        clear_output(wait=True)
        print(board)
        board.push(move)
        sleep(1)

In [53]:
# Function that returns list of position evaluations for entire game.
# Scores are in centipawn and from White perspective
# If value is None it's because engine saw checkmate in #n
def read_evals(game):
    
    eval_lst = []
    for node in game.mainline():
        if node.eval() != None:
            eval_lst.append(node.eval().white().score())

    return eval_lst

def read_moves(game):
    
    move_lst = [str(node.san()) for node in game.mainline()]
    return(move_lst)

def read_game_eval_recursive(game_node):
    
    if game_node.is_end():
        print(game_node.eval())
        return [None]
    else:
        if game_node.eval()!=None:
            return [game_node.eval().white().score()] + read_game_eval(game_node[0])
        else:
            return read_game_eval(game_node[0])   

In [56]:
class lichess_communication:
    
    lichess_id = None
    API_TOKEN = None
    client = None
    get_opening = True
    get_evals = True
    
    games_lst = []

    def __init__(self, user) -> None:
        
        # Initialize token
        with open('conf/token.txt') as f:
            self.API_TOKEN = f.readline()[:-1]
        
        # Initialize lichess client
        self.lichess_id = user
        session = berserk.TokenSession(self.API_TOKEN)
        self.client = berserk.Client(session=session)
    
    def fetch_games_by_dates(self, since, until):
    
        games_gen = self.client.games.export_by_player('miguel0f',
                                                    since=since,
                                                    until=until,
                                                    evals=self.get_evals,
                                                    opening=self.get_opening)
        
        self.games_lst = [GameModel(**game) for game in games_gen] 

    
        #for game in games_gen:
        #    try:
        #        self.games_lst.append(GameModel(**game))
        #    except:
        #        print(game)
            
        
    def get_games_info(self):
        
        if self.games_lst == None:
            print('No games have been fetched')
            return False
        else:
            print(f'Fetched {len(self.games_lst)} games')
            print(f'Last game from  : {self.games_lst[0]["createdAt"]}')
            print(f'First game from : {self.games_lst[-1]["createdAt"]}')

## API game request settings

In [57]:
last_Xdays = datetime.now()-timedelta(days=200)
since = int(berserk.utils.to_millis(last_Xdays))
until = int(berserk.utils.to_millis(datetime.now()))

lichess_comm = lichess_communication('miguel0f')
games_lst = lichess_comm.fetch_games_by_dates(since,until)

In [60]:
lichess_comm.games_lst[0].analysis

[MoveAnalysis(mate=None, eval=33, best=None, variation=None, judgment=None),
 MoveAnalysis(mate=None, eval=12, best=None, variation=None, judgment=None),
 MoveAnalysis(mate=None, eval=19, best=None, variation=None, judgment=None),
 MoveAnalysis(mate=None, eval=123, best='b8c6', variation='Nc6 Bb5 Nf6 d3 Bc5 c3 O-O O-O d5 Nbd2', judgment=Judgment(name='Inaccuracy', comment='Inaccuracy. Nc6 was best.')),
 MoveAnalysis(mate=None, eval=130, best=None, variation=None, judgment=None),
 MoveAnalysis(mate=None, eval=140, best=None, variation=None, judgment=None),
 MoveAnalysis(mate=None, eval=0, best='d1e2', variation='Qe2 Nf6 Nc3 Be7 Nxe4 O-O Nxf6+ Bxf6 d4 Qxd5', judgment=Judgment(name='Mistake', comment='Mistake. Qe2 was best.')),
 MoveAnalysis(mate=None, eval=5, best=None, variation=None, judgment=None),
 MoveAnalysis(mate=None, eval=13, best=None, variation=None, judgment=None),
 MoveAnalysis(mate=None, eval=35, best=None, variation=None, judgment=None),
 MoveAnalysis(mate=None, eval=9, be

In [ ]:
lichess_comm.get_games_info()

In [ ]:
lichess_comm.games_lst[0]

In [ ]:
games_df = {'lichess_id': [],
            'opening': [],
            'color':    [],
            'result':  [],
            'time':    [],
            'moves':   [],
            'evals':    [],
           'oponent': []}

for game in games_lst:   
    
    game_pgn = read_pgn(game)
    
    if game_pgn==None:
        break
    
    games_df['lichess_id'].append(game_pgn.headers['Site'][20:])
    
    games_df['opening'].append(game_pgn.headers['Opening'])
    games_df['color'].append('white' if game_pgn.headers['White']==user_lichess else 'black')
    
    if  game_pgn.headers['Result']=='1/2-1/2':
        games_df['result'].append('draw')
    elif game_pgn.headers['Result']=='1-0' and game_pgn.headers['White']==user_lichess:
        games_df['result'].append('win')
    elif game_pgn.headers['Result']=='0-1' and game_pgn.headers['Black']==user_lichess:
        games_df['result'].append('win')
    else:
        games_df['result'].append('loss')
    
    games_df['time'].append(game_pgn.headers['TimeControl'])
    
    games_df['moves'].append(read_moves(game_pgn))
    games_df['evals'].append(read_evals(game_pgn))
    
    if game_pgn.headers['White']==user_lichess:
        games_df['oponent'].append(game_pgn.headers['Black'])
    else:
        games_df['oponent'].append(game_pgn.headers['White'])
    
games_df = pd.DataFrame.from_dict(games_df, orient='columns')

In [ ]:
games_df['result'].value_counts()

In [ ]:
games_df.loc[games_df['result']=='loss']

In [ ]:
game = read_pgn(games_lst[12])
game_eval = games_df.iloc[12]
play_game(game)

plt.figure()
plt.plot(game_eval)
plt.show()

## Code snippets to look into data returned by API

In [ ]:
for k in range(len(a)):
    if a[k]['players'][color]['user']['name']!='miguel0f':
        print('ahahah')